In [ ]:
# Phase 1

# 1.1

## given diabetic_data.csv file into a Pandas Dataframe
import pandas as pd
import numpy as np

df = pd.read_csv('data_files/diabetic_data.csv')

## Initial check

df.head()
df.info()
df.describe()
df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

In [ ]:
#1.2

##  process the dataframe to interpret ? as a standard NumPy NaN value.
import pandas as pd
df = pd.read_csv('data_files/diabetic_data.csv', na_values='?')

if df['weight'].isnull().mean() > 0.90:
    df.drop(columns=['weight'], inplace=True)
    print("Dropped")
    

Dropped


C:\Users\Dell\AppData\Local\Temp\ipykernel_26504\2099501224.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data_files/diabetic_data.csv', na_values='?')


In [ ]:
#1.3

# read IDS_mapping file
mapping_df = pd.read_csv('data_files/IDs_mapping.csv' ,skiprows=10, nrows=31)

# marge df and mapping_df file
merged_df = pd.merge(df, mapping_df, on='discharge_disposition_id', how='left')

#find indexs for expired
expired_index = merged_df[
    merged_df['description'].str.contains('Expired', case=False, na=False)
].index

#before drop check df length
before = len(df)
#drop have expired rows and save df
df = merged_df.drop(expired_index)

#check new length of df
after = len(df)

print("Before:", before)
print("After:", after)
#check removed rows
print("Removed:", before - after)


Before: 101766
After: 100114
Removed: 1652


In [ ]:
#1.4

#Deduplication Strategy

#check for exact duplicate rows and count total number
duplicate_count = df.duplicated().sum()
print(duplicate_count)

#before drop check df length
before_drop = len(df)
#Remove the exact duplicate rows and assign the cleaned DataFrame back to the variable 'df'.
df = df.drop_duplicates(inplace=True)
##before drop check df length
after_drop = len(df)
#check removed rows
print(before_drop - after_drop)


0


TypeError: object of type 'NoneType' has no len()